In [1]:
## mmrotate1.x기반으로 진행 
# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)
# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

/usr/local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.0.0rc1
3.1.0
11.1
GCC 7.3


In [2]:
## config LOAD
from mmengine.config import Config
cfg = Config.fromfile('custom_CONFIG_HJS.py')

In [3]:
from mmdet.apis import init_detector, inference_detector
import mmrotate

config_file = 'custom_CONFIG_HJS.py'
checkpoint_file = 'ver1/epoch_100.pth'
model = init_detector(config_file, checkpoint_file, device='cuda:0')

Loads checkpoint by local backend from path: ver1/epoch_100.pth


In [4]:
## inference 진행시에 test pipeline 거치기 때문에 test pipeline을 학습시에도 미리 set해두기 (5120,5120)
## 계획 ==> 이미지를 1024 patch로 640 stride로 inference
from mmrotate.apis import inference_detector_by_patches

import cv2
img = cv2.imread('container_dataset/valid_images/OBJ00607_PS3_K3_NIA0086.png')


results=inference_detector_by_patches(model, img, [1024], [640], [1.0], dict(type='nms_rotated', iou_threshold=0.1))

print(results)
# Let's plot the result
# show_result_pyplot(model, img, result, score_thr=0.3, palette='dota')

<DetDataSample(

    META INFORMATION
    img_id: 0
    batch_input_shape: (5120, 5120)
    img_path: None
    ori_shape: (1024, 1024)
    img_shape: (5120, 5120)
    scale_factor: (5.0, 5.0)
    pad_shape: (5120, 5120)

    DATA FIELDS
    pred_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [5]:
import math
import numpy as np
def convert_xywha_to_8coords(xywha, is_clockwise=True):
    x, y, w, h, a = xywha
    angle = a if is_clockwise else -a

    lt_x, lt_y = -w / 2, -h / 2
    rt_x, rt_y = w / 2, - h/ 2
    rb_x, rb_y = w / 2, h / 2
    lb_x, lb_y = - w / 2, h / 2

    lt_x_ = lt_x * math.cos(angle) - lt_y * math.sin(angle) + x
    lt_y_ = lt_x * math.sin(angle) + lt_y * math.cos(angle) + y
    rt_x_ = rt_x * math.cos(angle) - rt_y * math.sin(angle) + x
    rt_y_ = rt_x * math.sin(angle) + rt_y * math.cos(angle) + y
    lb_x_ = lb_x * math.cos(angle) - lb_y * math.sin(angle) + x
    lb_y_ = lb_x * math.sin(angle) + lb_y * math.cos(angle) + y
    rb_x_ = rb_x * math.cos(angle) - rb_y * math.sin(angle) + x
    rb_y_ = rb_x * math.sin(angle) + rb_y * math.cos(angle) + y

    return [lt_x_, lt_y_, rt_x_, rt_y_, rb_x_, rb_y_, lb_x_, lb_y_]

In [6]:
test_r = results.pred_instances.bboxes[0].cpu().numpy()

In [7]:
test_pos = convert_xywha_to_8coords(test_r)
test_pos = list(map(int,test_pos))
test_pos = np.array([[test_pos[0:2]], [test_pos[2:4]], [test_pos[4:6]], [test_pos[6:8]]], np.int32)
print(test_pos)


[[[459 334]]

 [[475 339]]

 [[473 346]]

 [[457 341]]]


In [8]:
img = cv2.polylines(img, [test_pos], True, (255, 255, 255), 2)

In [9]:
cv2.imwrite('check.png',img)

True